# Práctica 4. Similitud de palabras y oraciones

## Generar cuerpo de documentos

In [1]:
import os
import nltk
from nltk.corpus import PlaintextCorpusReader
import re

carpeta_libros = "Libros"

# Leer todos los libros en la carpeta y guardarlos en una lista
documentos = []
for nombre_archivo in os.listdir(carpeta_libros):
    if nombre_archivo.endswith('.txt'):
        with open(os.path.join(carpeta_libros, nombre_archivo), 'r', encoding='utf-8') as archivo:
            documentos.append(archivo.read())

In [2]:
# Lista de stopwords
stopwords = set(nltk.corpus.stopwords.words('english'))
# Crear el corpus
corpus = PlaintextCorpusReader(carpeta_libros, '.*\.txt')

file_ids = corpus.fileids()
file_ids

['Dreams - Henri Bergson.txt',
 'Memoirs of Extraordinary Popular Delusions and the Madness of Crowds - Charles Mackay.txt',
 'Nerves and Common Sense - Annie Payson Call.txt',
 'Ten Thousand Dreams Interpreted - Gustavus Hindman Miller.txt',
 'The Mind and the Brain - Alfred Binet.txt',
 'The Mind of the Child - William T. Preyer.txt',
 'The Science of Human Nature A Psychology for Beginners - William Henry Pyle.txt',
 'The Trained Memory - Warren Hilton.txt',
 'The Untroubled Mind - Herbert J. Hall.txt',
 'Unconscious Memory - Samuel Butler.txt']

## Preparación de texto

In [3]:
# Funciones de preprocesamiento
def remove_stopwords(text):
    return [word for word in text if word not in stopwords]

def lematizar(text):
    lemmatizer = nltk.WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in text]

def tokenizar(text):
    lowercase_text = text.lower()
    cleaned_text = re.sub(r'[^a-z]', ' ', lowercase_text) # Eliminar caracteres no alfabéticos
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text) # Eliminar espacios en blanco
    
    sent_text = nltk.sent_tokenize(cleaned_text) # Obtener oraciones
    all_tagged_text = []
    # iterar sobre cada oración y preprocesarla
    for sentence in sent_text:
        tokenized_text = nltk.word_tokenize(sentence)
        tokenized_text = remove_stopwords(tokenized_text)
        lemma_text = lematizar(tokenized_text)
        tagged_text = nltk.pos_tag(lemma_text)
        all_tagged_text.extend(tagged_text)
    return all_tagged_text


## Similitud de Palabras con synsets

### Simulitud de verbos con métrica "path_similarity"

In [4]:
from collections import Counter
from nltk.corpus import wordnet as wn
# Funcion para obtener los synsets de un verbo
def obtener_verbo_synsets(verb):
    return wn.synsets(verb, pos=wn.VERB)
# Funcion para calcular la similitud entre dos verbos
def calcular_similitud_path(verb1, verb2):
    synsets1 = obtener_verbo_synsets(verb1)
    synsets2 = obtener_verbo_synsets(verb2)
    max_similarity = 0 # Similaridad máxima
    for synset1 in synsets1:
        for synset2 in synsets2:
            similarity = synset1.path_similarity(synset2) # Calcular la similitud entre los synsets
            if similarity and similarity > max_similarity: 
                max_similarity = similarity # Actualizar la similaridad máxima
    return max_similarity
# Funcion para encontrar los verbos más similares a un verbo dado
def encontrar_verbos_similares(most_common_verb, verbos):
    similaridades = [] # Lista de similaridades
    for verbo in set(verbos):
        if verbo != most_common_verb:
            similarity = calcular_similitud_path(most_common_verb, verbo) # Calcular la similitud
            if similarity:  # Ignorar similaridades None
                similaridades.append((verbo, similarity)) 
    similaridades.sort(key=lambda x: x[1], reverse=True) # Ordenar por similaridad
    return similaridades[:5] # Devolver los 5 verbos más similares
# Iterar sobre los documentos
for doc in corpus.fileids():
    print(doc)
    verbos = []
    text = corpus.raw(doc)
    tagged_text = tokenizar(text)
    #Obtener los verbos
    for word, tag in tagged_text: 
        if tag == 'VB' or tag == 'VBD' or tag == 'VBG' or tag == 'VBN' or tag == 'VBP' or tag == 'VBZ':
            verbos.append(word)
    #Obtener el verbo más común
    counter = Counter(verbos)
    most_common_verb = counter.most_common(1)
    print("Most common verb: ",most_common_verb)
    #Obtener los synsets del verbo más común
    verbos_similares= encontrar_verbos_similares(most_common_verb[0][0], verbos)
    print(verbos_similares, "\n")
    


Dreams - Henri Bergson.txt
Most common verb:  [('dreaming', 4)]
[('dream', 1.0), ('felt', 0.3333333333333333), ('thought', 0.3333333333333333), ('found', 0.3333333333333333), ('seen', 0.3333333333333333)] 

Memoirs of Extraordinary Popular Delusions and the Madness of Crowds - Charles Mackay.txt
Most common verb:  [('made', 30)]
[('held', 1.0), ('established', 1.0), ('take', 1.0), ('created', 1.0), ('threw', 1.0)] 

Nerves and Common Sense - Annie Payson Call.txt
Most common verb:  [('come', 3)]
[('see', 0.5), ('grow', 0.5), ('seeing', 0.5), ('form', 0.3333333333333333), ('assured', 0.3333333333333333)] 

Ten Thousand Dreams Interpreted - Gustavus Hindman Miller.txt
Most common verb:  [('thought', 38)]
[('mean', 1.0), ('intended', 1.0), ('thinking', 1.0), ('recall', 1.0), ('think', 1.0)] 

The Mind and the Brain - Alfred Binet.txt
Most common verb:  [('thought', 5)]
[('mean', 1.0), ('supposed', 1.0), ('consider', 1.0), ('meaning', 1.0), ('suppose', 1.0)] 

The Mind of the Child - Willi

### Similitud de sustantivos con métrica 'path_similarity'

In [5]:
#MIsmo procedimiento pero con sustantivos
def obtener_noun_synsets(noun):
    return wn.synsets(noun, pos=wn.NOUN)

def calcular_similitud_path(noun1, noun2):
    synsets1 = obtener_noun_synsets(noun1)
    synsets2 = obtener_noun_synsets(noun2)
    max_similarity = 0
    for synset1 in synsets1:
        for synset2 in synsets2:
            similarity = synset1.path_similarity(synset2)
            if similarity and similarity > max_similarity:
                max_similarity = similarity
    return max_similarity

def encontrar_nouns_similares(most_common_noun, nouns):
    similaridades = []
    for noun in set(nouns):
        if noun != most_common_noun:
            similarity = calcular_similitud_path(most_common_noun, noun)
            if similarity:  # Ignorar similaridades None
                similaridades.append((noun, similarity))
    similaridades.sort(key=lambda x: x[1], reverse=True)
    return similaridades[:5]

for doc in corpus.fileids():
    print(doc)
    nouns = []
    text = corpus.raw(doc)
    tagged_text = tokenizar(text)
    for word, tag in tagged_text: 
        if tag == 'NN' or tag == 'NNS' or tag == 'NNP' or tag == 'NNPS':
            nouns.append(word)
    #Obtener el sustantivo más común
    counter = Counter(nouns)
    most_common_noun = counter.most_common(1)
    print("Most common noun: ",most_common_noun)
    #Obtener los synsets del sustantivo más común
    nouns_similares = encontrar_nouns_similares(most_common_noun[0][0], nouns)
    print(nouns_similares, "\n")

Dreams - Henri Bergson.txt
Most common noun:  [('dream', 20)]
[('ambition', 1.0), ('imagery', 0.5), ('desire', 0.5), ('vision', 0.5), ('state', 0.3333333333333333)] 

Memoirs of Extraordinary Popular Delusions and the Madness of Crowds - Charles Mackay.txt
Most common noun:  [('law', 124)]
[('police', 1.0), ('philosophy', 0.5), ('posse', 0.5), ('force', 0.5), ('collection', 0.5)] 

Nerves and Common Sense - Annie Payson Call.txt
Most common noun:  [('habit', 7)]
[('change', 0.25), ('act', 0.2), ('mind', 0.2), ('power', 0.2), ('turn', 0.2)] 

Ten Thousand Dreams Interpreted - Gustavus Hindman Miller.txt
Most common noun:  [('dream', 171)]
[('aspiration', 1.0), ('reverie', 0.5), ('vision', 0.5), ('imagination', 0.5), ('desire', 0.5)] 

The Mind and the Brain - Alfred Binet.txt
Most common noun:  [('mind', 15)]
[('idea', 1.0), ('opinion', 0.5), ('content', 0.3333333333333333), ('place', 0.3333333333333333), ('position', 0.25)] 

The Mind of the Child - William T. Preyer.txt
Most common no

### Similitud de verbos con métrica 'wup_similarity'

In [6]:
#MIsmo procedimiento pero con la similitud de Wup (Wu-Palmer)
def obtener_verbo_synsets(verb):
    return wn.synsets(verb, pos=wn.VERB)

def calcular_similitud_wup(verb1, verb2):
    synsets1 = obtener_verbo_synsets(verb1)
    synsets2 = obtener_verbo_synsets(verb2)
    max_similarity = 0
    for synset1 in synsets1:
        for synset2 in synsets2:
            similarity = synset1.wup_similarity(synset2) # Calcular la similitud entre los synsets
            if similarity and similarity > max_similarity:
                max_similarity = similarity
    return max_similarity

def encontrar_verbos_similares(most_common_verb, verbos):
    similaridades = []
    for verbo in set(verbos):
        if verbo != most_common_verb:
            similarity = calcular_similitud_wup(most_common_verb, verbo)
            if similarity:  # Ignorar similaridades None
                similaridades.append((verbo, similarity))
    similaridades.sort(key=lambda x: x[1], reverse=True)
    return similaridades[:5]
# Iterar sobre los documentos
for doc in corpus.fileids():
    print(doc)
    verbos = []
    text = corpus.raw(doc)
    tagged_text = tokenizar(text)
    for word, tag in tagged_text: 
        if tag == 'VB' or tag == 'VBD' or tag == 'VBG' or tag == 'VBN' or tag == 'VBP' or tag == 'VBZ':
            verbos.append(word)
    #Obtener el verbo más común
    counter = Counter(verbos)
    most_common_verb = counter.most_common(1)
    print("Most common verb: ",most_common_verb)
    #Obtener los synsets del verbo más común
    verbos_similares= encontrar_verbos_similares(most_common_verb[0][0], verbos)
    print(verbos_similares, "\n")

Dreams - Henri Bergson.txt
Most common verb:  [('dreaming', 4)]
[('dream', 1.0), ('thought', 0.75), ('seen', 0.75), ('see', 0.75), ('think', 0.75)] 

Memoirs of Extraordinary Popular Delusions and the Madness of Crowds - Charles Mackay.txt
Most common verb:  [('made', 30)]
[('held', 1.0), ('established', 1.0), ('take', 1.0), ('created', 1.0), ('threw', 1.0)] 

Nerves and Common Sense - Annie Payson Call.txt
Most common verb:  [('come', 3)]
[('see', 0.8571428571428571), ('seeing', 0.8571428571428571), ('grow', 0.8), ('known', 0.75), ('suffer', 0.75)] 

Ten Thousand Dreams Interpreted - Gustavus Hindman Miller.txt
Most common verb:  [('thought', 38)]
[('mean', 1.0), ('intended', 1.0), ('thinking', 1.0), ('recall', 1.0), ('think', 1.0)] 

The Mind and the Brain - Alfred Binet.txt
Most common verb:  [('thought', 5)]
[('mean', 1.0), ('supposed', 1.0), ('consider', 1.0), ('meaning', 1.0), ('suppose', 1.0)] 

The Mind of the Child - William T. Preyer.txt
Most common verb:  [('made', 15)]
[('e

### Similitid de sustantivos con 'wup_similarity'

In [7]:
# MIsmo procedimiento pero con sustantivos y wu palmer
def obtener_noun_synsets(noun):
    return wn.synsets(noun, pos=wn.NOUN)

def calcular_similitud_wup(noun1, noun2):
    synsets1 = obtener_noun_synsets(noun1)
    synsets2 = obtener_noun_synsets(noun2)
    max_similarity = 0
    for synset1 in synsets1:
        for synset2 in synsets2:
            similarity = synset1.wup_similarity(synset2)
            if similarity and similarity > max_similarity:
                max_similarity = similarity
    return max_similarity

def encontrar_nouns_similares(most_common_noun, nouns):
    similaridades = []
    for noun in set(nouns):
        if noun != most_common_noun:
            similarity = calcular_similitud_wup(most_common_noun, noun)
            if similarity:  # Ignorar similaridades None
                similaridades.append((noun, similarity))
    similaridades.sort(key=lambda x: x[1], reverse=True)
    return similaridades[:5]

for doc in corpus.fileids():
    print(doc)
    nouns = []
    text = corpus.raw(doc)
    tagged_text = tokenizar(text)
    for word, tag in tagged_text: 
        if tag == 'NN' or tag == 'NNS' or tag == 'NNP' or tag == 'NNPS':
            nouns.append(word)
    #Obtener el sustantivo más común
    counter = Counter(nouns)
    most_common_noun = counter.most_common(1)
    print("Most common noun: ",most_common_noun)
    #Obtener los synsets del sustantivo más común
    nouns_similares = encontrar_nouns_similares(most_common_noun[0][0], nouns)
    print(nouns_similares, "\n")

Dreams - Henri Bergson.txt
Most common noun:  [('dream', 20)]
[('ambition', 1.0), ('imagery', 0.9411764705882353), ('vision', 0.9333333333333333), ('desire', 0.9230769230769231), ('death', 0.8235294117647058)] 

Memoirs of Extraordinary Popular Delusions and the Madness of Crowds - Charles Mackay.txt
Most common noun:  [('law', 124)]
[('philosophy', 0.9473684210526315), ('principle', 0.9411764705882353), ('force', 0.9230769230769231), ('police', 0.9), ('office', 0.8888888888888888)] 

Nerves and Common Sense - Annie Payson Call.txt
Most common noun:  [('habit', 7)]
[('change', 0.8421052631578947), ('turn', 0.75), ('work', 0.75), ('act', 0.7142857142857143), ('trouble', 0.7058823529411765)] 

Ten Thousand Dreams Interpreted - Gustavus Hindman Miller.txt
Most common noun:  [('dream', 171)]
[('aspiration', 1.0), ('reverie', 0.9523809523809523), ('nightmare', 0.9473684210526315), ('imagination', 0.9411764705882353), ('vision', 0.9333333333333333)] 

The Mind and the Brain - Alfred Binet.tx

## Similtud de documentos con synsets

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer 
from rake_nltk import Rake
# Se elige la primera frase clave de cada documento con RAKE-NLTK
#Se carga el modelo de RAKE
rake_nltk_var= Rake()
# Función para obtener oraciones
def oraciones(text):
    lowercase_text = text.lower()
    cleaned_text = re.sub(r'[^a-z]', ' ', lowercase_text) # Eliminar caracteres no alfabéticos
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text) # Eliminar espacios en blanco
    
    sent_text = nltk.sent_tokenize(cleaned_text) # Obtener oraciones
    return sent_text

# Obtener la frase clave más importante de cada documento con RAKE
frases = []
# Iterar sobre los documentos
for doc in corpus.fileids():
    print(doc)
    text = corpus.raw(doc)
    sent_text = oraciones(text)
    texto=' '.join(sent_text)
    #Extraer las frases clave
    rake_nltk_var.extract_keywords_from_text(texto)
    #Obtener la frase clave más importante
    keyword_extracted = rake_nltk_var.get_ranked_phrases()[0]
    print('Frase más representativa:', keyword_extracted, "\n")
    # Guardar la frase clave
    frases.append(keyword_extracted)

Dreams - Henri Bergson.txt
Palabra clave más importante: essay professor bergson made several contributions 

Memoirs of Extraordinary Popular Delusions and the Madness of Crowds - Charles Mackay.txt
Palabra clave más importante: que lass arriva dans notre bonne ville monsieur le r gent publia que lass serait utile pour r tablir la nation la faridondaine la faridondon mais il nous 

Nerves and Common Sense - Annie Payson Call.txt
Palabra clave más importante: faith cure christian science mind cure hypnotism psychotherapeutics 

Ten Thousand Dreams Interpreted - Gustavus Hindman Miller.txt
Palabra clave más importante: virile serpents cankerous lizards slimy intestine worms hairy 

The Mind and the Brain - Alfred Binet.txt
Palabra clave más importante: hitherto dreamed first let us say 

The Mind of the Child - William T. Preyer.txt
Palabra clave más importante: deaf mutes grunds tze und grundz ge zur aufstellung eines lehrplans f r eine taubstummen anstalt 

The Science of Human Nature

### Similitud con métrica 'path_similarity'

In [22]:
#Se obtiene los synsets de una palabra
def obtener_synsets(palabra):
    return wn.synsets(palabra)
#Se calcula la similitud entre dos oraciones
def calcular_similitud_oraciones(phrase1, phrase2):  
    # Tokenizar las frases
    words1 = phrase1.split()
    words2 = phrase2.split()  
    max_similarities = []
    # Iterar sobre las palabras de la primera frase
    for word1 in words1:
        # Obtener los synsets de la palabra
        synsets1 = obtener_synsets(word1)
        max_similarity = 0
        for word2 in words2:
            # Obtener los synsets de la palabra de la segunda frase
            synsets2 = obtener_synsets(word2)
            for synset1 in synsets1:
                for synset2 in synsets2:
                    # Calcular la similitud entre los synsets
                    similarity = synset1.path_similarity(synset2)
                    if similarity and similarity > max_similarity:
                        max_similarity = similarity
        if max_similarity > 0:
            # Guardar la similitud máxima
            max_similarities.append(max_similarity)
    # Calcular la similitud promedio
    if max_similarities:
        return sum(max_similarities) / len(max_similarities)
    else:
        return 0
    
#La frase a comparar es la numero 7
frase_base = frases[6]
print(f"Frase base: {frase_base}\n")
similitudes = []
# Iterar sobre las frases de los documentos
for i, frase in enumerate(frases):
    if i != 6:
        # Calcular la similitud entre la frase base y la frase actual
        similitud = calcular_similitud_oraciones(frase_base, frase)
        similitudes.append((i+1, similitud))

# Imprimir las similitudes
print(f"Similitudes de la frase del documento 7 con las demás frases:")
for doc_num, similitud in similitudes:
    print(f"Documento {doc_num}: Similitud = {similitud:.4f}")

Frase base: certain well defined situations certain events always take place according

Similitudes de la frase del documento 7 con las demás frases:
Documento 1: Similitud = 0.3733
Documento 2: Similitud = 0.3083
Documento 3: Similitud = 0.3060
Documento 4: Similitud = 0.2893
Documento 5: Similitud = 0.2950
Documento 6: Similitud = 0.2893
Documento 8: Similitud = 0.3083
Documento 9: Similitud = 0.2917
Documento 10: Similitud = 0.3010


## Similitud de palabras con embedding

In [9]:
import gensim.downloader as api

# Cargar el modelo GloVe
glove_model = api.load("glove-wiki-gigaword-300")
# Función para obtener los términos más similares a una palabra con GloVe
def obtener_terminos_similares(glove_model, word, top_n=5):
    try:
        similar_terms = glove_model.most_similar(word, topn=top_n)
        return similar_terms
    except KeyError:
        return []
# Iterar sobre los documentos
for doc in corpus.fileids():
    print(doc)
    verbos = []
    text = corpus.raw(doc)
    tagged_text = tokenizar(text)
    #Obtener los verbos
    for word, tag in tagged_text: 
        if tag == 'VB' or tag == 'VBD' or tag == 'VBG' or tag == 'VBN' or tag == 'VBP' or tag == 'VBZ':
            verbos.append(word)
    #Obtener el verbo más común
    counter = Counter(verbos)
    most_common_verb = counter.most_common(1)
    print("Most common verb: ",most_common_verb)
    #Obtener los términos similares del verbo más común
    if most_common_verb:
        # Obtener los términos más similares
        similares = obtener_terminos_similares(glove_model, most_common_verb, top_n=5)
        print(f"Los 5 términos más similares a '{most_common_verb}' son:")
        for termino, similitud in similares:
            print(f"Término: {termino}, Similitud: {similitud}")
    print("\n")

Dreams - Henri Bergson.txt
Most common verb:  [('dreaming', 4)]
Los 5 términos más similares a '[('dreaming', 4)]' son:
Término: dreamed, Similitud: 0.5849594473838806
Término: dreams, Similitud: 0.5801305770874023
Término: dream, Similitud: 0.5788585543632507
Término: waking, Similitud: 0.5402055978775024
Término: dreamt, Similitud: 0.5081028938293457


Memoirs of Extraordinary Popular Delusions and the Madness of Crowds - Charles Mackay.txt
Most common verb:  [('made', 30)]
Los 5 términos más similares a '[('made', 30)]' son:
Término: making, Similitud: 0.8200170397758484
Término: make, Similitud: 0.7930080890655518
Término: but, Similitud: 0.6358116865158081
Término: came, Similitud: 0.6152335405349731
Término: that, Similitud: 0.6122232675552368


Nerves and Common Sense - Annie Payson Call.txt
Most common verb:  [('come', 3)]
Los 5 términos más similares a '[('come', 3)]' son:
Término: coming, Similitud: 0.8334023952484131
Término: go, Similitud: 0.7987576127052307
Término: n't, S

## Similitud de documentos con embedding

In [26]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

# Cargar el tokenizer y el modelo BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def obtener_embedding(frase):
    # Tokenizar la frase y convertirla a tensores de entrada
    inputs = tokenizer(frase, return_tensors='pt', truncation=True, padding=True, max_length=512)
    
    # Obtener la salida de BERT
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Obtener el embedding de la [CLS] token
    cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()
    
    return cls_embedding

# Obtener los embeddings de las frases
embeddings = [obtener_embedding(frase) for frase in frases]

# Embedding del documento 7 (índice 6 si se usa una lista 0-indexada)
embedding_doc7 = embeddings[6]

# Calcular la similitud de coseno entre el documento 7 y los demás documentos
similitud_doc7 = cosine_similarity([embedding_doc7], embeddings)

# Imprimir las similitudes
for i, similitud in enumerate(similitud_doc7[0]):
    if i == 6:
        continue  # Saltar la comparación consigo mismo
    print(f"Similitud entre el documento 7 y el documento {i + 1}: {similitud:.4f}")

Similitud entre el documento 7 y el documento 1: 0.8381
Similitud entre el documento 7 y el documento 2: 0.7777
Similitud entre el documento 7 y el documento 3: 0.8196
Similitud entre el documento 7 y el documento 4: 0.7747
Similitud entre el documento 7 y el documento 5: 0.8654
Similitud entre el documento 7 y el documento 6: 0.8007
Similitud entre el documento 7 y el documento 8: 0.8829
Similitud entre el documento 7 y el documento 9: 0.8580
Similitud entre el documento 7 y el documento 10: 0.8742
